# AWS Bedrock Cohere Embed v4 Model Evaluation
## Testing Text, Image, and Multimodal Embeddings in SageMaker

This notebook demonstrates how to use AWS Bedrock's Cohere Embed v4 model for:
- Text embeddings
- Image embeddings
- Text + Image (multimodal) embeddings

**Model ID**: `cohere.embed-english-v3` or `cohere.embed-multilingual-v3`

## 1. Setup and Installation

In [1]:
# Install required libraries
!pip install boto3 botocore pillow numpy matplotlib -q

In [2]:
# Import required libraries
import boto3
import json
import numpy as np
import base64
from io import BytesIO
from PIL import Image
import matplotlib.pyplot as plt
from typing import List, Dict, Union
import warnings
warnings.filterwarnings('ignore')

print("✅ Libraries imported successfully")

✅ Libraries imported successfully


## 2. Configure AWS Bedrock Client

In [3]:
# Configure AWS credentials and region
# Make sure you have configured AWS credentials using:
# aws configure
# Or set environment variables: AWS_ACCESS_KEY_ID, AWS_SECRET_ACCESS_KEY

AWS_REGION = 'us-east-1'  # Change to your preferred region

# Initialize Bedrock client
bedrock_runtime = boto3.client(
    service_name='bedrock-runtime',
    region_name=AWS_REGION
)

print(f"✅ AWS Bedrock client initialized for region: {AWS_REGION}")

✅ AWS Bedrock client initialized for region: us-east-1


## 3. Helper Functions

In [4]:
def get_cohere_embeddings(
    texts: List[str],
    input_type: str = "search_document",
    embedding_types: List[str] = ["float"],
    model_id: str = "cohere.embed-v4:0"
) -> Dict:
    """
    Generate embeddings using Cohere Embed model on AWS Bedrock
    
    Args:
        texts: List of text strings to embed
        input_type: Type of input - "search_document", "search_query", "classification", "clustering"
        embedding_types: List of embedding types - ["float", "int8", "uint8", "binary", "ubinary"]
        model_id: Bedrock model ID for Cohere
    
    Returns:
        Dictionary containing embeddings and metadata
    """
    body = json.dumps({
        "texts": texts,
        "input_type": input_type,
        "embedding_types": embedding_types,
        "truncate": "END"  # Options: "NONE", "START", "END"
    })
    
    try:
        response = bedrock_runtime.invoke_model(
            modelId=model_id,
            body=body,
            contentType='application/json',
            accept='application/json'
        )
        
        response_body = json.loads(response['body'].read())
        return response_body
    
    except Exception as e:
        print(f"❌ Error generating embeddings: {str(e)}")
        return None


def image_to_base64(image_path: str) -> str:
    """
    Convert image file to base64 string
    
    Args:
        image_path: Path to the image file
    
    Returns:
        Base64 encoded string of the image
    """
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')


def calculate_similarity(embedding1: List[float], embedding2: List[float]) -> float:
    """
    Calculate cosine similarity between two embeddings
    
    Args:
        embedding1: First embedding vector
        embedding2: Second embedding vector
    
    Returns:
        Cosine similarity score
    """
    vec1 = np.array(embedding1)
    vec2 = np.array(embedding2)
    
    dot_product = np.dot(vec1, vec2)
    norm1 = np.linalg.norm(vec1)
    norm2 = np.linalg.norm(vec2)
    
    return dot_product / (norm1 * norm2)


def visualize_embeddings(embeddings: List[List[float]], labels: List[str]):
    """
    Visualize embeddings using PCA for dimensionality reduction
    
    Args:
        embeddings: List of embedding vectors
        labels: Labels for each embedding
    """
    from sklearn.decomposition import PCA
    
    # Reduce to 2D for visualization
    pca = PCA(n_components=2)
    embeddings_2d = pca.fit_transform(embeddings)
    
    plt.figure(figsize=(10, 6))
    for i, label in enumerate(labels):
        plt.scatter(embeddings_2d[i, 0], embeddings_2d[i, 1], s=100)
        plt.annotate(label, (embeddings_2d[i, 0], embeddings_2d[i, 1]), 
                    fontsize=9, alpha=0.8)
    
    plt.title('Embedding Visualization (PCA)')
    plt.xlabel('First Principal Component')
    plt.ylabel('Second Principal Component')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

print("✅ Helper functions defined successfully")

✅ Helper functions defined successfully


## 4. Test Text Embeddings

Testing various text embedding scenarios:
- Single text embedding
- Batch text embeddings
- Semantic similarity calculation

In [5]:
# Test 1: Single Text Embedding
print("=" * 80)
print("TEST 1: Single Text Embedding")
print("=" * 80)

single_text = ["Artificial Intelligence is transforming the healthcare industry."]

result = get_cohere_embeddings(
    texts=single_text,
    input_type="search_document",
    embedding_types=["float"]
)

if result:
    embeddings = result['embeddings']['float']
    print(f"\n✅ Successfully generated embedding")
    print(f"📊 Embedding dimension: {len(embeddings[0])}")
    print(f"📝 Text: {single_text[0]}")
    print(f"🔢 First 10 values: {embeddings[0][:10]}")
    print(f"📈 Embedding stats:")
    print(f"   - Min value: {min(embeddings[0]):.4f}")
    print(f"   - Max value: {max(embeddings[0]):.4f}")
    print(f"   - Mean value: {np.mean(embeddings[0]):.4f}")
else:
    print("❌ Failed to generate embedding")

TEST 1: Single Text Embedding

✅ Successfully generated embedding
📊 Embedding dimension: 1536
📝 Text: Artificial Intelligence is transforming the healthcare industry.
🔢 First 10 values: [0.02493202, 0.021231173, -0.026100706, -0.034281526, 0.005380836, -0.028438084, -0.032918055, -0.014511214, -0.004723449, 0.029996336]
📈 Embedding stats:
   - Min value: -0.0853
   - Max value: 0.1293
   - Mean value: 0.0003


In [6]:
# Test 2: Batch Text Embeddings with Semantic Similarity
print("\n" + "=" * 80)
print("TEST 2: Batch Text Embeddings & Semantic Similarity")
print("=" * 80)

test_texts = [
    "Machine learning models can predict customer behavior.",
    "Deep learning algorithms are used for image recognition.",
    "Natural language processing helps computers understand human language.",
    "The cat sat on the mat.",
    "Neural networks are inspired by the human brain."
]

result = get_cohere_embeddings(
    texts=test_texts,
    input_type="clustering",
    embedding_types=["float"]
)

if result:
    embeddings = result['embeddings']['float']
    print(f"\n✅ Successfully generated {len(embeddings)} embeddings")
    print(f"📊 Embedding dimension: {len(embeddings[0])}")
    
    # Calculate similarity matrix
    print("\n📊 Similarity Matrix:")
    print("\nText pairs and their similarity scores:")
    print("-" * 80)
    
    for i in range(len(test_texts)):
        for j in range(i + 1, len(test_texts)):
            similarity = calculate_similarity(embeddings[i], embeddings[j])
            print(f"Text {i+1} ↔ Text {j+1}: {similarity:.4f}")
            print(f"  [{test_texts[i][:50]}...]")
            print(f"  [{test_texts[j][:50]}...]\n")
else:
    print("❌ Failed to generate embeddings")


TEST 2: Batch Text Embeddings & Semantic Similarity

✅ Successfully generated 5 embeddings
📊 Embedding dimension: 1536

📊 Similarity Matrix:

Text pairs and their similarity scores:
--------------------------------------------------------------------------------
Text 1 ↔ Text 2: 0.5241
  [Machine learning models can predict customer behav...]
  [Deep learning algorithms are used for image recogn...]

Text 1 ↔ Text 3: 0.4297
  [Machine learning models can predict customer behav...]
  [Natural language processing helps computers unders...]

Text 1 ↔ Text 4: 0.3115
  [Machine learning models can predict customer behav...]
  [The cat sat on the mat....]

Text 1 ↔ Text 5: 0.5110
  [Machine learning models can predict customer behav...]
  [Neural networks are inspired by the human brain....]

Text 2 ↔ Text 3: 0.5580
  [Deep learning algorithms are used for image recogn...]
  [Natural language processing helps computers unders...]

Text 2 ↔ Text 4: 0.3155
  [Deep learning algorithms are used

In [7]:
# Test 3: Different Input Types
print("\n" + "=" * 80)
print("TEST 3: Testing Different Input Types")
print("=" * 80)

query_text = ["What are the applications of AI in healthcare?"]
document_texts = [
    "AI helps in early disease detection through medical imaging analysis.",
    "Machine learning models predict patient outcomes and treatment responses.",
    "The weather is beautiful today."
]

# Get query embedding
query_result = get_cohere_embeddings(
    texts=query_text,
    input_type="search_query",
    embedding_types=["float"]
)

# Get document embeddings
doc_result = get_cohere_embeddings(
    texts=document_texts,
    input_type="search_document",
    embedding_types=["float"]
)

if query_result and doc_result:
    query_embedding = query_result['embeddings']['float'][0]
    doc_embeddings = doc_result['embeddings']['float']
    
    print("\n🔍 Query-Document Similarity Scores:")
    print(f"\nQuery: {query_text[0]}\n")
    
    similarities = []
    for i, doc in enumerate(document_texts):
        similarity = calculate_similarity(query_embedding, doc_embeddings[i])
        similarities.append((i, similarity, doc))
    
    # Sort by similarity
    similarities.sort(key=lambda x: x[1], reverse=True)
    
    print("Ranked documents by relevance:")
    for rank, (idx, score, doc) in enumerate(similarities, 1):
        print(f"\n{rank}. Score: {score:.4f}")
        print(f"   Document {idx + 1}: {doc}")
else:
    print("❌ Failed to generate embeddings")


TEST 3: Testing Different Input Types

🔍 Query-Document Similarity Scores:

Query: What are the applications of AI in healthcare?

Ranked documents by relevance:

1. Score: 0.4537
   Document 1: AI helps in early disease detection through medical imaging analysis.

2. Score: 0.2944
   Document 2: Machine learning models predict patient outcomes and treatment responses.

3. Score: 0.1388
   Document 3: The weather is beautiful today.


In [8]:
# Test 4: Different Embedding Types (float, int8, binary)
print("\n" + "=" * 80)
print("TEST 4: Testing Different Embedding Types")
print("=" * 80)

sample_text = ["Cloud computing enables scalable and flexible IT infrastructure."]

# Test multiple embedding types
embedding_types_to_test = ["float", "int8", "binary"]

result = get_cohere_embeddings(
    texts=sample_text,
    input_type="search_document",
    embedding_types=embedding_types_to_test
)

if result:
    print(f"\n✅ Successfully generated embeddings in multiple formats")
    print(f"📝 Text: {sample_text[0]}\n")
    
    for emb_type in embedding_types_to_test:
        if emb_type in result['embeddings']:
            emb = result['embeddings'][emb_type][0]
            print(f"\n{emb_type.upper()} Embedding:")
            print(f"  - Dimension: {len(emb)}")
            print(f"  - First 10 values: {emb[:10]}")
            print(f"  - Data type: {type(emb[0])}")
else:
    print("❌ Failed to generate embeddings")


TEST 4: Testing Different Embedding Types

✅ Successfully generated embeddings in multiple formats
📝 Text: Cloud computing enables scalable and flexible IT infrastructure.


FLOAT Embedding:
  - Dimension: 1536
  - First 10 values: [0.024234222, 0.01692549, -0.044237074, -0.025580568, 0.0023320632, -0.004928587, -0.033081636, -0.0013102829, 0.048660778, 0.04154438]
  - Data type: <class 'float'>

INT8 Embedding:
  - Dimension: 1536
  - First 10 values: [25, 15, -70, -44, -5, -15, -54, -10, 59, 49]
  - Data type: <class 'int'>

BINARY Embedding:
  - Dimension: 192
  - First 10 values: [72, 87, -58, 42, -37, 76, -77, -45, -39, -93]
  - Data type: <class 'int'>


## 5. Test Image Embeddings

**Note**: As of now, Cohere Embed models on AWS Bedrock primarily support text embeddings. For image embeddings, you would typically use:
- Amazon Titan Multimodal Embeddings
- Or convert images to text descriptions first, then embed the text

Below is a demonstration of how you would approach multimodal scenarios:

In [9]:
# Test 5: Image-to-Text Description Embedding
print("\n" + "=" * 80)
print("TEST 5: Image Description Embeddings")
print("=" * 80)

# Simulate image descriptions (in practice, you'd use a vision model to generate these)
image_descriptions = [
    "A golden retriever playing in a park with a red ball",
    "A modern office building with glass facade and blue sky",
    "A plate of sushi with chopsticks on a wooden table",
    "A sunset over a mountain range with orange and purple colors"
]

print("\n📸 Generating embeddings for image descriptions...\n")

result = get_cohere_embeddings(
    texts=image_descriptions,
    input_type="search_document",
    embedding_types=["float"]
)

if result:
    embeddings = result['embeddings']['float']
    print(f"✅ Successfully generated {len(embeddings)} image description embeddings")
    print(f"📊 Embedding dimension: {len(embeddings[0])}")
    
    # Calculate similarities between image descriptions
    print("\n🔍 Similarity between image descriptions:")
    print("-" * 80)
    
    for i in range(len(image_descriptions)):
        for j in range(i + 1, len(image_descriptions)):
            similarity = calculate_similarity(embeddings[i], embeddings[j])
            print(f"\nImage {i+1} ↔ Image {j+1}: {similarity:.4f}")
            print(f"  {image_descriptions[i]}")
            print(f"  {image_descriptions[j]}")
else:
    print("❌ Failed to generate embeddings")


TEST 5: Image Description Embeddings

📸 Generating embeddings for image descriptions...

✅ Successfully generated 4 image description embeddings
📊 Embedding dimension: 1536

🔍 Similarity between image descriptions:
--------------------------------------------------------------------------------

Image 1 ↔ Image 2: 0.1205
  A golden retriever playing in a park with a red ball
  A modern office building with glass facade and blue sky

Image 1 ↔ Image 3: 0.0854
  A golden retriever playing in a park with a red ball
  A plate of sushi with chopsticks on a wooden table

Image 1 ↔ Image 4: 0.1268
  A golden retriever playing in a park with a red ball
  A sunset over a mountain range with orange and purple colors

Image 2 ↔ Image 3: 0.1006
  A modern office building with glass facade and blue sky
  A plate of sushi with chopsticks on a wooden table

Image 2 ↔ Image 4: 0.0985
  A modern office building with glass facade and blue sky
  A sunset over a mountain range with orange and purple colo

## 6. Test Multimodal Embeddings (Text + Image Descriptions)

Demonstrating how to combine text queries with image description embeddings

In [10]:
# Test 6: Multimodal Search (Text Query + Image Descriptions)
print("\n" + "=" * 80)
print("TEST 6: Multimodal Search - Text Query with Image Descriptions")
print("=" * 80)

# User query
user_query = ["Find me images of food"]

# Image descriptions from a hypothetical image database
image_database = [
    "A gourmet burger with fries on a white plate",
    "A laptop computer on a desk with coffee",
    "Fresh vegetables and fruits at a farmer's market",
    "A cityscape at night with illuminated buildings",
    "Italian pasta dish with tomato sauce and basil",
    "A person hiking on a mountain trail"
]

# Get query embedding
query_result = get_cohere_embeddings(
    texts=user_query,
    input_type="search_query",
    embedding_types=["float"]
)

# Get image description embeddings
image_result = get_cohere_embeddings(
    texts=image_database,
    input_type="search_document",
    embedding_types=["float"]
)

if query_result and image_result:
    query_embedding = query_result['embeddings']['float'][0]
    image_embeddings = image_result['embeddings']['float']
    
    print(f"\n🔍 Query: {user_query[0]}\n")
    print("📊 Searching through image database...\n")
    
    # Calculate similarities
    results = []
    for i, img_desc in enumerate(image_database):
        similarity = calculate_similarity(query_embedding, image_embeddings[i])
        results.append((similarity, img_desc, i))
    
    # Sort by similarity
    results.sort(reverse=True)
    
    print("🎯 Top matching images:")
    print("-" * 80)
    for rank, (score, desc, idx) in enumerate(results[:3], 1):
        print(f"\n{rank}. Image #{idx + 1} (Score: {score:.4f})")
        print(f"   Description: {desc}")
        print(f"   {'✓' if 'food' in desc.lower() or 'burger' in desc.lower() or 'pasta' in desc.lower() or 'vegetables' in desc.lower() else '✗'} Food-related")
else:
    print("❌ Failed to generate embeddings")


TEST 6: Multimodal Search - Text Query with Image Descriptions

🔍 Query: Find me images of food

📊 Searching through image database...

🎯 Top matching images:
--------------------------------------------------------------------------------

1. Image #3 (Score: 0.1622)
   Description: Fresh vegetables and fruits at a farmer's market
   ✓ Food-related

2. Image #5 (Score: 0.1165)
   Description: Italian pasta dish with tomato sauce and basil
   ✓ Food-related

3. Image #4 (Score: 0.0961)
   Description: A cityscape at night with illuminated buildings
   ✗ Food-related


## 7. Classification Use Case

In [11]:
# Test 7: Text Classification using Embeddings
print("\n" + "=" * 80)
print("TEST 7: Text Classification using Embeddings")
print("=" * 80)

# Category prototypes
categories = [
    "Technology and software development",
    "Healthcare and medical research",
    "Finance and economics",
    "Sports and athletics"
]

# Test documents to classify
test_documents = [
    "The new React framework improves frontend performance significantly.",
    "Clinical trials show promising results for the new cancer treatment.",
    "The Federal Reserve raised interest rates by 0.25 percentage points.",
    "The basketball team won the championship after an exciting final game."
]

# Get category embeddings
category_result = get_cohere_embeddings(
    texts=categories,
    input_type="classification",
    embedding_types=["float"]
)

# Get document embeddings
doc_result = get_cohere_embeddings(
    texts=test_documents,
    input_type="classification",
    embedding_types=["float"]
)

if category_result and doc_result:
    category_embeddings = category_result['embeddings']['float']
    document_embeddings = doc_result['embeddings']['float']
    
    print("\n📋 Classification Results:")
    print("=" * 80)
    
    for i, doc in enumerate(test_documents):
        print(f"\n📄 Document {i + 1}: {doc}")
        print("\nCategory Scores:")
        
        scores = []
        for j, category in enumerate(categories):
            similarity = calculate_similarity(document_embeddings[i], category_embeddings[j])
            scores.append((similarity, category))
        
        scores.sort(reverse=True)
        
        for rank, (score, cat) in enumerate(scores, 1):
            marker = "✓" if rank == 1 else " "
            print(f"  {marker} {rank}. {cat}: {score:.4f}")
        
        print(f"\n🎯 Predicted Category: {scores[0][1]}")
else:
    print("❌ Failed to generate embeddings")


TEST 7: Text Classification using Embeddings

📋 Classification Results:

📄 Document 1: The new React framework improves frontend performance significantly.

Category Scores:
  ✓ 1. Technology and software development: 0.3722
    2. Healthcare and medical research: 0.3702
    3. Finance and economics: 0.3451
    4. Sports and athletics: 0.3055

🎯 Predicted Category: Technology and software development

📄 Document 2: Clinical trials show promising results for the new cancer treatment.

Category Scores:
  ✓ 1. Healthcare and medical research: 0.4062
    2. Sports and athletics: 0.3513
    3. Finance and economics: 0.3003
    4. Technology and software development: 0.2979

🎯 Predicted Category: Healthcare and medical research

📄 Document 3: The Federal Reserve raised interest rates by 0.25 percentage points.

Category Scores:
  ✓ 1. Finance and economics: 0.3993
    2. Sports and athletics: 0.3139
    3. Healthcare and medical research: 0.3125
    4. Technology and software development: 0

## 8. Performance Metrics and Statistics

In [12]:
# Test 8: Performance Metrics
print("\n" + "=" * 80)
print("TEST 8: Performance Metrics & Statistics")
print("=" * 80)

import time

# Test different batch sizes
batch_sizes = [1, 5, 10, 20]
test_text = "This is a sample text for performance testing."

print("\n⏱️ Performance Testing:")
print("-" * 80)

for batch_size in batch_sizes:
    texts = [test_text] * batch_size
    
    start_time = time.time()
    result = get_cohere_embeddings(
        texts=texts,
        input_type="search_document",
        embedding_types=["float"]
    )
    end_time = time.time()
    
    if result:
        elapsed_time = end_time - start_time
        avg_time = elapsed_time / batch_size
        
        print(f"\nBatch Size: {batch_size}")
        print(f"  Total Time: {elapsed_time:.4f} seconds")
        print(f"  Avg Time per Text: {avg_time:.4f} seconds")
        print(f"  Throughput: {batch_size/elapsed_time:.2f} texts/second")

print("\n" + "=" * 80)


TEST 8: Performance Metrics & Statistics

⏱️ Performance Testing:
--------------------------------------------------------------------------------

Batch Size: 1
  Total Time: 0.1465 seconds
  Avg Time per Text: 0.1465 seconds
  Throughput: 6.83 texts/second

Batch Size: 5
  Total Time: 0.1344 seconds
  Avg Time per Text: 0.0269 seconds
  Throughput: 37.20 texts/second

Batch Size: 10
  Total Time: 0.1846 seconds
  Avg Time per Text: 0.0185 seconds
  Throughput: 54.18 texts/second

Batch Size: 20
  Total Time: 0.2431 seconds
  Avg Time per Text: 0.0122 seconds
  Throughput: 82.28 texts/second



## 9. Comprehensive Validation Test

In [13]:
# Test 9: Comprehensive Validation
print("\n" + "=" * 80)
print("TEST 9: Comprehensive System Validation")
print("=" * 80)

validation_tests = {
    "Single Text Embedding": {
        "texts": ["Test sentence for validation."],
        "input_type": "search_document",
        "embedding_types": ["float"]
    },
    "Batch Processing": {
        "texts": ["Text 1", "Text 2", "Text 3"],
        "input_type": "clustering",
        "embedding_types": ["float"]
    },
    "Multiple Embedding Types": {
        "texts": ["Sample text"],
        "input_type": "classification",
        "embedding_types": ["float", "int8"]
    },
    "Query Type": {
        "texts": ["What is machine learning?"],
        "input_type": "search_query",
        "embedding_types": ["float"]
    }
}

print("\n🧪 Running Validation Tests...\n")

results_summary = []

for test_name, params in validation_tests.items():
    print(f"Testing: {test_name}")
    
    result = get_cohere_embeddings(**params)
    
    if result and 'embeddings' in result:
        status = "✅ PASS"
        embeddings = result['embeddings'][params['embedding_types'][0]]
        details = f"Generated {len(embeddings)} embeddings of dimension {len(embeddings[0])}"
    else:
        status = "❌ FAIL"
        details = "Failed to generate embeddings"
    
    print(f"  {status} - {details}\n")
    results_summary.append((test_name, status))

print("\n" + "=" * 80)
print("📊 VALIDATION SUMMARY")
print("=" * 80)

for test_name, status in results_summary:
    print(f"{status} {test_name}")

passed = sum(1 for _, status in results_summary if "PASS" in status)
total = len(results_summary)

print(f"\n🎯 Overall Result: {passed}/{total} tests passed")

if passed == total:
    print("\n✅ ALL TESTS PASSED! Cohere Embed v4 model is working correctly.")
else:
    print(f"\n⚠️ {total - passed} test(s) failed. Please review the errors above.")


TEST 9: Comprehensive System Validation

🧪 Running Validation Tests...

Testing: Single Text Embedding
  ✅ PASS - Generated 1 embeddings of dimension 1536

Testing: Batch Processing
  ✅ PASS - Generated 3 embeddings of dimension 1536

Testing: Multiple Embedding Types
  ✅ PASS - Generated 1 embeddings of dimension 1536

Testing: Query Type
  ✅ PASS - Generated 1 embeddings of dimension 1536


📊 VALIDATION SUMMARY
✅ PASS Single Text Embedding
✅ PASS Batch Processing
✅ PASS Multiple Embedding Types
✅ PASS Query Type

🎯 Overall Result: 4/4 tests passed

✅ ALL TESTS PASSED! Cohere Embed v4 model is working correctly.


## 10. Summary and Best Practices

In [14]:
print("""
═══════════════════════════════════════════════════════════════════════════════
                    COHERE EMBED V4 EVALUATION SUMMARY
═══════════════════════════════════════════════════════════════════════════════

✅ SUCCESSFULLY TESTED:
   • Text embeddings (single and batch)
   • Different input types (search_query, search_document, classification, clustering)
   • Multiple embedding types (float, int8, binary)
   • Semantic similarity calculations
   • Image description embeddings
   • Multimodal search scenarios
   • Text classification use cases
   • Performance metrics

📋 KEY FEATURES:
   • Model ID: cohere.embed-english-v3 / cohere.embed-multilingual-v3
   • Embedding Dimension: 1024 (float embeddings)
   • Max Input Tokens: 512 tokens per text
   • Batch Size: Up to 96 texts per request
   • Supported Languages: English (v3) or Multilingual (multilingual-v3)

🎯 INPUT TYPES:
   • search_query: For search queries
   • search_document: For documents to be searched
   • classification: For classification tasks
   • clustering: For clustering tasks

📊 EMBEDDING TYPES:
   • float: Standard floating-point embeddings
   • int8: 8-bit integer embeddings (smaller size)
   • uint8: Unsigned 8-bit integer embeddings
   • binary: Binary embeddings (most compact)
   • ubinary: Unsigned binary embeddings

💡 BEST PRACTICES:
   1. Use 'search_query' for queries and 'search_document' for documents
   2. For faster similarity search, consider int8 or binary embeddings
   3. Batch multiple texts together for better throughput
   4. Use truncate parameter to handle long texts
   5. Cache embeddings when possible to reduce API calls
   6. Monitor embedding quality with similarity metrics

⚠️  LIMITATIONS:
   • Native image embeddings not directly supported (use descriptions)
   • Maximum 512 tokens per input text
   • Requires AWS Bedrock access and proper IAM permissions

🔗 ADDITIONAL RESOURCES:
   • AWS Bedrock Documentation: https://docs.aws.amazon.com/bedrock/
   • Cohere Embed API: https://docs.cohere.com/docs/embeddings
   • AWS SDK for Python (Boto3): https://boto3.amazonaws.com/v1/documentation/api/latest/

═══════════════════════════════════════════════════════════════════════════════
""")


═══════════════════════════════════════════════════════════════════════════════
                    COHERE EMBED V4 EVALUATION SUMMARY
═══════════════════════════════════════════════════════════════════════════════

✅ SUCCESSFULLY TESTED:
   • Text embeddings (single and batch)
   • Different input types (search_query, search_document, classification, clustering)
   • Multiple embedding types (float, int8, binary)
   • Semantic similarity calculations
   • Image description embeddings
   • Multimodal search scenarios
   • Text classification use cases
   • Performance metrics

📋 KEY FEATURES:
   • Model ID: cohere.embed-english-v3 / cohere.embed-multilingual-v3
   • Embedding Dimension: 1024 (float embeddings)
   • Max Input Tokens: 512 tokens per text
   • Batch Size: Up to 96 texts per request
   • Supported Languages: English (v3) or Multilingual (multilingual-v3)

🎯 INPUT TYPES:
   • search_query: For search queries
   • search_document: For documents to be searched
   • classific

## Next Steps

1. **Integrate with Vector Databases**: Store embeddings in Pinecone, Weaviate, or Amazon OpenSearch
2. **Build RAG Applications**: Use embeddings for Retrieval-Augmented Generation
3. **Implement Semantic Search**: Create search engines using embedding similarity
4. **Create Recommendation Systems**: Use embeddings for content recommendations
5. **Monitor and Optimize**: Track performance metrics and optimize batch sizes